# **Preprocessing Data**

**Mục tiêu: chuẩn bị dữ liệu sạch** để có thể đưa vào modeling:
   - Xử lý giá trị thiếu (missing values) cho cả numeric và categorical.
   - Nhận diện & xử lý outlier cho một số biến số quan trọng.
   - Chuẩn hóa / biến đổi thang đo (scaling) cho các cột numeric.
   - Thực hiện một số **feature engineering** dựa trên insight từ EDA.
   - Mã hóa (encoding) các thuộc tính ordinal & categorical.
   - Xây dựng **ma trận đặc trưng `X`** và nhãn `y` sẵn sàng cho modeling.

## Import thư viện & module trong src

In [1]:
import os
import sys
import numpy as np

PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
if PROJECT_ROOT not in sys.path:
    sys.path.append(PROJECT_ROOT)

from src.data_processing import (
    load_csv_as_str,
    summarize_missing,
    string_column_to_float,
    impute_numeric,
    impute_categorical,
    detect_outliers_iqr,
    clip_outliers_iqr,
    zscore_standardize,
    experience_to_numeric,
    last_new_job_to_numeric,
    log_transform,
    EDUCATION_LEVEL_ORDER,
    ENROLLED_UNI_ORDER,
    EXPERIENCE_ORDER,
    COMPANY_SIZE_ORDER,
    LAST_NEW_JOB_ORDER,
    ordinal_encode,
    one_hot_encode,
    frequency_encode,
)

np.set_printoptions(suppress=True)

DATA_PATH   = "../data/raw/aug_train.csv"
OUTPUT_DIR  = "../data/processed"
OUTPUT_FILE = "aug_train.csv"   

## Load dữ liệu

In [2]:
header, data_raw = load_csv_as_str(
    DATA_PATH,
    delimiter=",",
    has_header=True,
    encoding="utf-8",
)

print("Số dòng (samples):", data_raw.shape[0])
print("Số cột  (features):", data_raw.shape[1])

col_index = {name: i for i, name in enumerate(header)}
n_rows = data_raw.shape[0]

Số dòng (samples): 19158
Số cột  (features): 14


## II. Thống kê Missing Values

Mục tiêu:

- Đếm số lượng và tỷ lệ missing cho từng cột.
- Từ đó quyết định chiến lược xử lý missing cho numeric / ordinal / categorical.

Hàm `summarize_missing` sẽ:
- Chuẩn hóa các token như `""`, `" "`, `"NA"`, `"nan"`, `"null"`, `"?"`… về missing.
- Trả về:
  - `missing_counts`: số ô missing theo cột.
  - `missing_ratios`: tỷ lệ missing theo cột.

In [3]:
missing_counts, missing_ratios = summarize_missing(data_raw, header=header, extra_tokens=None)

Missing summary theo column:
enrollee_id               |      0 missing ( 0.00%)
city                      |      0 missing ( 0.00%)
city_development_index    |      0 missing ( 0.00%)
gender                    |   4508 missing (23.53%)
relevent_experience       |      0 missing ( 0.00%)
enrolled_university       |    386 missing ( 2.01%)
education_level           |    460 missing ( 2.40%)
major_discipline          |   2813 missing (14.68%)
experience                |     65 missing ( 0.34%)
company_size              |   5938 missing (30.99%)
company_type              |   6140 missing (32.05%)
last_new_job              |    423 missing ( 2.21%)
training_hours            |      0 missing ( 0.00%)
target                    |      0 missing ( 0.00%)


## III. Xử lý các biến Numeric

### **1. Parse string sang float**

Ý tưởng:

- Cột trong CSV đang là chuỗi.
- Dùng `string_column_to_float` để:
  - Chuẩn hóa chuỗi,
  - Chuyển thành `float`,
  - Missing → `np.nan` (không tự ý điền).


In [4]:
city_col = data_raw[:, col_index["city_development_index"]]
train_col = data_raw[:, col_index["training_hours"]]

city_float = string_column_to_float(city_col)
train_float = string_column_to_float(train_col)

N = 10

city_raw = city_col[:N]
train_raw = train_col[:N]

city_clean = city_float[:N]
train_clean = train_float[:N]

print("\t\t\t\tSo sánh trước và sau khi xử lí")
print("=" * 100)
print(f"{'Index':<5} | {'city (raw)':<25} | {'city (float)':<20} | {'train (raw)':<20} | {'train (float)':<15}")
print("-" * 100)

for i in range(N):
    print(f"{i:<5} | {city_raw[i]:<25} | {city_clean[i]:<20} | {train_raw[i]:<20} | {train_clean[i]:<15}")

print("=" * 100)

				So sánh trước và sau khi xử lí
Index | city (raw)                | city (float)         | train (raw)          | train (float)  
----------------------------------------------------------------------------------------------------
0     | 0.92                      | 0.92                 | 36                   | 36.0           
1     | 0.7759999999999999        | 0.7759999999999999   | 47                   | 47.0           
2     | 0.624                     | 0.624                | 83                   | 83.0           
3     | 0.789                     | 0.789                | 52                   | 52.0           
4     | 0.767                     | 0.767                | 8                    | 8.0            
5     | 0.764                     | 0.764                | 24                   | 24.0           
6     | 0.92                      | 0.92                 | 24                   | 24.0           
7     | 0.762                     | 0.762                | 18                   

### **2. Impute missing cho numeric**

Ý tưởng:

- Với dataset này 2 cột numeric **không có missing**, nhưng để pipeline tổng quát:
  - Dùng `impute_numeric` với strategy `"median"`.
- Nếu có missing:
  - Tính median trên các giá trị hợp lệ,
  - Điền `np.nan` bằng median.


In [5]:
city_imputed, city_fill = impute_numeric(city_col, strategy="median")
train_imputed, train_fill = impute_numeric(train_col, strategy="median")

print("Giá trị điền cho city_development_index:", city_fill)
print("Giá trị điền cho training_hours:", train_fill)

Giá trị điền cho city_development_index: 0.903
Giá trị điền cho training_hours: 47.0


### **3. Phát hiện & xử lý outlier (IQR)**

**Cơ sở từ EDA:**
Biến `training_hours` có phân phối lệch phải và chứa lượng lớn ngoại lai (~5% dữ liệu). Đây là các giá trị thực (valid data), không phải nhiễu, nên không được xóa bỏ mà cần xử lý để tránh làm lệch mô hình (đặc biệt là KNN).

**Chiến lược:**
1. **Dùng phương pháp IQR:** Phù hợp với dữ liệu phân phối không chuẩn.
   - Tính giới hạn: `Lower = Q1 - 1.5*IQR`, `Upper = Q3 + 1.5*IQR`.
   - Xác định điểm dữ liệu nằm ngoài khoảng này là outlier.

2. **Xử lý bằng Clipping (Winsorization):**
   - Thay vì loại bỏ dòng dữ liệu, ta **kẹp (clip)** các giá trị vượt ngưỡng về mốc `Upper` hoặc `Lower`.
   - **Tác dụng:** Giảm thiểu ảnh hưởng của cực trị lên việc tính toán khoảng cách/trọng số của mô hình, đồng thời bảo toàn kích thước mẫu dữ liệu.

**Hàm sử dụng:**
- `detect_outliers_iqr`
- `clip_outliers_iqr`

In [6]:
# Phát hiện outliers
city_out_mask  = detect_outliers_iqr(city_imputed)
train_out_mask = detect_outliers_iqr(train_imputed)

# Số outlier
n_city_out   = int(city_out_mask.sum())
n_train_out  = int(train_out_mask.sum())

# Min/Max trước clip
city_min_before, city_max_before   = float(city_imputed.min()),  float(city_imputed.max())
train_min_before, train_max_before = float(train_imputed.min()), float(train_imputed.max())

# Thực hiện clip
cdi_clip  = clip_outliers_iqr(city_imputed)
train_clip = clip_outliers_iqr(train_imputed)

# Min/Max sau clip
city_min_after, city_max_after   = float(cdi_clip.min()),  float(cdi_clip.max())
train_min_after, train_max_after = float(train_clip.min()), float(train_clip.max())

print("KẾT QUẢ XỬ LÍ OUTLIERS (IQR MethodMethod):")

# Tổng số outlier
print("\n1.  Số lượng outlier phát hiện:")
print(f"   - city_development_index : {n_city_out}")
print(f"   - training_hours         : {n_train_out}")

# So sánh Before/After
print("\n2.  So sánh min/max TRƯỚC và SAU khi clip:")
print(f"   - city_development_index : Before = ({city_min_before:.4f}, {city_max_before:.4f})   |   After = ({city_min_after:.4f}, {city_max_after:.4f})")
print(f"   - training_hours         : Before = ({train_min_before:.4f}, {train_max_before:.4f}) |   After = ({train_min_after:.4f}, {train_max_after:.4f})")

KẾT QUẢ XỬ LÍ OUTLIERS (IQR MethodMethod):

1.  Số lượng outlier phát hiện:
   - city_development_index : 17
   - training_hours         : 984

2.  So sánh min/max TRƯỚC và SAU khi clip:
   - city_development_index : Before = (0.4480, 0.9490)   |   After = (0.4700, 0.9490)
   - training_hours         : Before = (1.0000, 336.0000) |   After = (1.0000, 185.5000)


### **4. Biến đổi phân phối & chuẩn hóa**

Ý tưởng:

- `training_hours` **lệch phải mạnh** nên ta sẽ dùng `log_transform`:
  - Giảm skew, đưa phân phối gần chuẩn hơn.
- Sau đó, chuẩn hóa **Z-score** cho cả 2 feature `training_hours` và `city_development_index`:
  - `x_std = (x - mean) / std`.
  - Giúp các feature numeric nằm trên cùng thang đo (trung bình 0, độ lệch chuẩn ≈ 1).

In [7]:
# log-transform cho training_hours
train_log = log_transform(train_clip)

# Z-score cho cả 2 biến
cdi_z,  city_mean,  cdi_std  = zscore_standardize(cdi_clip)
train_z, train_mean, train_std = zscore_standardize(train_log)

# Kiểm tra mean/std sau standardize
cdi_z_mean,  cdi_z_std  = float(np.mean(cdi_z)),  float(np.std(cdi_z))
train_z_mean, train_z_std = float(np.mean(train_z)), float(np.std(train_z))

# Min/Max trước transform
city_min0, city_max0 = float(cdi_clip.min()),  float(cdi_clip.max())
train_min0, train_max0 = float(train_clip.min()), float(train_clip.max())

# Min/Max sau log-transform
train_log_min, train_log_max = float(train_log.min()), float(train_log.max())

# Min/Max sau Z-score
cdi_z_min, cdi_z_max = float(cdi_z.min()), float(cdi_z.max())
train_z_min, train_z_max = float(train_z.min()), float(train_z.max())

print("Kết quả sau khi log-transform và z-score:")
print("\n1. So sánh trước và sau khi log-transform (training_hours):")
print(f"   Before : min = {train_min0:.4f}, max = {train_max0:.4f}")
print(f"   After  : min = {train_log_min:.4f}, max = {train_log_max:.4f}")

print("\n2. So sánh min/max trước và sau khi z-score:")
print(f"   city_development_index : before = ({city_min0:.4f}, {city_max0:.4f}), "
      f"after = ({cdi_z_min:.4f}, {cdi_z_max:.4f})")
print(f"   training_hours (log)   : before = ({train_log_min:.4f}, {train_log_max:.4f}), "
      f"after = ({train_z_min:.4f}, {train_z_max:.4f})")

print("\n3. Mean/std sau khi z-score:")
print(f"   city_development_index : mean = {cdi_z_mean:.4f}, std = {cdi_z_std:.4f}")
print(f"   training_hours (log)   : mean = {train_z_mean:.4f}, std = {train_z_std:.4f}")

print("\n4. Tham số gốc dùng để z-score (mean/std):")
print(f"   city_development_index : mean = {city_mean:.4f}, std = {cdi_std:.4f}")
print(f"   training_hours (log)   : mean = {train_mean:.4f}, std = {train_std:.4f}")

Kết quả sau khi log-transform và z-score:

1. So sánh trước và sau khi log-transform (training_hours):
   Before : min = 1.0000, max = 185.5000
   After  : min = 0.0000, max = 5.2231

2. So sánh min/max trước và sau khi z-score:
   city_development_index : before = (0.4700, 0.9490), after = (-2.9105, 0.9743)
   training_hours (log)   : before = (0.0000, 5.2231), after = (-3.8916, 1.5315)

3. Mean/std sau khi z-score:
   city_development_index : mean = -0.0000, std = 1.0000
   training_hours (log)   : mean = 0.0000, std = 1.0000

4. Tham số gốc dùng để z-score (mean/std):
   city_development_index : mean = 0.8289, std = 0.1233
   training_hours (log)   : mean = 3.7481, std = 0.9631


#### Nhận xét kết quả log-transform và z-score

- **Log-transform (training_hours)**: giá trị lớn được nén mạnh (max giảm từ ~185 xuống ~5), giúp giảm skew và giảm ảnh hưởng outlier. Đây là kết quả hoàn toàn bình thường.
- **Z-score**: sau chuẩn hóa, cả hai biến đều có mean ≈ 0 và std ≈ 1. Min/max sau z-score nằm trong khoảng (-4, 4), phù hợp với phân phối đã clip và log-transform.
- **Tham số gốc (mean/std)** phản ánh đúng phân bố ban đầu: city_development_index tập trung quanh 0.8, còn training_hours(log) nằm quanh ~3.7.

## IV. Feature thứ bậc (Ordinal Features)

Nhóm này được chia làm 2 loại dựa trên đặc tính dữ liệu: **Định danh có thứ tự** và **Bán định lượng**.

#### **1. Nhóm định danh có thứ tự (`education_level`, `company_size`, `enrolled_university`)**

**Chiến lược:** Sử dụng **Manual Mapping** (Ánh xạ thủ công) để đảm bảo đúng thứ bậc logic và giữ lại thông tin "Missing" như một tín hiệu rủi ro.

1.  **Xử lý Missing:** Điền `NaN` → `"Unknown"`.
2.  **Định nghĩa bộ từ điển (Mapping Dictionary):**
    * Gán `"Unknown"` = 0 (Để giữ tính thứ tự và không làm mất dữ liệu).
    * Các cấp bậc tiếp theo tăng dần (1, 2, 3...).

**Quy tắc Map cụ thể:**
* **`education_level`**: Unknown (0) < Primary (1) < High School (2) < Graduate (3) < Masters (4) < Phd (5).
* **`company_size`**: Unknown (0) < <10 (1) < ... < 10000+ (8).
* **`enrolled_university`**: no_enrollment (0) < Part time (1) < Full time (2).

#### **2. Nhóm bán định lượng (`experience`, `last_new_job`)**

**Chiến lược:** **Clean & Convert** (Làm sạch và chuyển sang số thực) để phản ánh đúng độ lớn thời gian.

1.  **Không dùng LabelEncoder:** Vì các chuỗi `>20` hay `never` cần được hiểu theo nghĩa toán học (độ lớn), không phải thứ tự bảng chữ cái.
2.  **Logic chuyển đổi (Custom Parsing):**
    * **`experience`**:
        * `<1` → 0
        * `>20` → 21
        * Giá trị số (`1`..`20`) → giữ nguyên `int`.
        * Missing → 0 (hoặc -1, tránh điền Median).
    * **`last_new_job`**:
        * `never` → 0
        * `>4` → 5
        * Missing → 0.
3.  **Xử lý sau chuyển đổi:**
    * Dữ liệu trở thành dạng **Numeric**.
    * Áp dụng **Scaling** (StandardScaler) nếu dùng mô hình khoảng cách (KNN, Logistic).

##### Lý do:
* **Bỏ bước `impute median`:** EDA chỉ ra nhóm Missing/Unknown có tỷ lệ churn ~40%. Điền Median sẽ làm mất tín hiệu quan trọng này.
* **Gán `Unknown = 0`:** Giữ nguyên bản chất "thiếu thông tin" thay vì gán ép vào một nhóm có sẵn.
* **Numeric Conversion:** Chuyển `experience` về số năm thực tế giúp mô hình hiểu khoảng cách (ví dụ: 20 năm kinh nghiệm > 1 năm kinh nghiệm gấp 20 lần) tốt hơn là gán index ngẫu nhiên.

### **Nhóm bán định lượng `experience` và `last_new_job`**

#### **Convert sang numeric**

In [8]:
exp_col = data_raw[:, col_index["experience"]]
lnj_col = data_raw[:, col_index["last_new_job"]]

exp_num_raw = experience_to_numeric(exp_col)
lnj_num_raw = last_new_job_to_numeric(lnj_col)

#### **Impute missing**

In [9]:
# Impute median cho experience_num
exp_num = exp_num_raw.copy()
mask_nan_exp = np.isnan(exp_num)
if np.any(mask_nan_exp):
    exp_median = np.nanmedian(exp_num)
    exp_num[mask_nan_exp] = exp_median
else:
    exp_median = None

# Impute median cho last_new_job_num
lnj_num = lnj_num_raw.copy()
mask_nan_lnj = np.isnan(lnj_num)
if np.any(mask_nan_lnj):
    lnj_median = np.nanmedian(lnj_num)
    lnj_num[mask_nan_lnj] = lnj_median
else:
    lnj_median = None

#### **Scale**

In [10]:
# Z-score
exp_num_z, exp_num_mean, exp_num_std = zscore_standardize(exp_num)
lnj_num_z, lnj_num_mean, lnj_num_std = zscore_standardize(lnj_num)

# Before values (min/max)
exp_before_min, exp_before_max = float(np.nanmin(exp_num_raw)), float(np.nanmax(exp_num_raw))
lnj_before_min, lnj_before_max = float(np.nanmin(lnj_num_raw)), float(np.nanmax(lnj_num_raw))

# After median impute
exp_after_min, exp_after_max = float(exp_num.min()), float(exp_num.max())
lnj_after_min, lnj_after_max = float(lnj_num.min()), float(lnj_num.max())

# Z-score min/max
exp_z_min, exp_z_max = float(exp_num_z.min()), float(exp_num_z.max())
lnj_z_min, lnj_z_max = float(lnj_num_z.min()), float(lnj_num_z.max())

# Before values (min/max trước khi z-score)
exp_before_z_min, exp_before_z_max = float(exp_num.min()), float(exp_num.max())
lnj_before_z_min, lnj_before_z_max = float(lnj_num.min()), float(lnj_num.max())

# After values (sau khi z-score)
exp_after_z_min, exp_after_z_max = float(exp_num_z.min()), float(exp_num_z.max())
lnj_after_z_min, lnj_after_z_max = float(lnj_num_z.min()), float(lnj_num_z.max())

print("Xử lí cho feature experience và last_new_job:")

# Median impute
print("\nSo sánh trước và sau khi impute median:")

print(f"   experience_num     : before = ({exp_before_min:.4f}, {exp_before_max:.4f}), "
      f"after = ({exp_after_min:.4f}, {exp_after_max:.4f}), "
      f"median_fill = {exp_median}")

print(f"   last_new_job_num   : before = ({lnj_before_min:.4f}, {lnj_before_max:.4f}), "
      f"after = ({lnj_after_min:.4f}, {lnj_after_max:.4f}), "
      f"median_fill = {lnj_median}")

# Z-score
print("\nSo sánh min/max trước và sau z-score:")

print(f"   experience_num     : before = ({exp_before_z_min:.4f}, {exp_before_z_max:.4f}), "
      f"after = ({exp_after_z_min:.4f}, {exp_after_z_max:.4f})")

print(f"   last_new_job_num   : before = ({lnj_before_z_min:.4f}, {lnj_before_z_max:.4f}), "
      f"after = ({lnj_after_z_min:.4f}, {lnj_after_z_max:.4f})")

# Mean/std sau z-score
print("\nMean/std sau khi z-score:")
print(f"   experience_num_z   : mean = {exp_num_z.mean():.4f}, std = {exp_num_z.std():.4f}")
print(f"   last_new_job_num_z : mean = {lnj_num_z.mean():.4f}, std = {lnj_num_z.std():.4f}")

# Tham số gốc
print("\nTham số gốc (mean/std):")
print(f"   experience_num     : mean = {exp_num_mean:.4f}, std = {exp_num_std:.4f}")
print(f"   last_new_job_num   : mean = {lnj_num_mean:.4f}, std = {lnj_num_std:.4f}")


Xử lí cho feature experience và last_new_job:

So sánh trước và sau khi impute median:
   experience_num     : before = (0.0000, 21.0000), after = (0.0000, 21.0000), median_fill = 9.0
   last_new_job_num   : before = (0.0000, 5.0000), after = (0.0000, 5.0000), median_fill = 1.0

So sánh min/max trước và sau z-score:
   experience_num     : before = (0.0000, 21.0000), after = (-1.4923, 1.6116)
   last_new_job_num   : before = (0.0000, 5.0000), after = (-1.1893, 1.8165)

Mean/std sau khi z-score:
   experience_num_z   : mean = 0.0000, std = 1.0000
   last_new_job_num_z : mean = 0.0000, std = 1.0000

Tham số gốc (mean/std):
   experience_num     : mean = 10.0964, std = 6.7656
   last_new_job_num   : mean = 1.9783, std = 1.6635


### **Nhóm định danh có thứ tự**

#### **Impute missing**

In [11]:
edu_col   = data_raw[:, col_index["education_level"]]
csize_col = data_raw[:, col_index["company_size"]]
enr_col   = data_raw[:, col_index["enrolled_university"]]

edu_filled,   edu_fill_val   = impute_categorical(edu_col,   strategy="constant", constant_value="Unknown")
csize_filled, csize_fill_val = impute_categorical(csize_col, strategy="constant", constant_value="Unknown")
enr_filled,   enr_fill_val   = impute_categorical(enr_col,   strategy="constant", constant_value="Unknown")

print("education_level fill :", edu_fill_val)
print("company_size fill   :", csize_fill_val)
print("enrolled_university :", enr_fill_val)

education_level fill : Unknown
company_size fill   : Unknown
enrolled_university : Unknown


#### **Ordinal encode**

In [12]:
edu_ord,   edu_map   = ordinal_encode(edu_filled,   EDUCATION_LEVEL_ORDER)
csize_ord, cs_map    = ordinal_encode(csize_filled, COMPANY_SIZE_ORDER)
enr_ord,   enr_map   = ordinal_encode(enr_filled,   ENROLLED_UNI_ORDER)

## V. Biến nhiều nhóm (High Cardinality)

**Feature:**
- `city`

Chiến lược:
- **Không dùng One-Hot** vì sẽ tạo quá nhiều cột dư thừa.
- Dùng **Frequency Encoding**:
  - Tính tần suất xuất hiện (tỷ lệ %) của từng `city` trên tổng số mẫu.
  - Thay thế nhãn tên thành phố bằng giá trị tần suất tương ứng.
  - **Ý nghĩa:** Chuyển đổi dữ liệu từ dạng định danh sang dạng số thực, phản ánh mức độ "phổ biến" của thành phố đó trong tập dữ liệu (thành phố lớn $\to$ giá trị cao, thành phố nhỏ $\to$ giá trị thấp).
- Lý do chọn Frequency Encoding:

    1. **Tránh bùng nổ số chiều (Curse of Dimensionality):**
        - Biến `city` có 123 giá trị. Nếu dùng One-Hot sẽ tạo ra 123 cột dư thừa, gây chậm và loãng dữ liệu. Frequency Encoding gói gọn chỉ trong **1 cột**.

    2. **Tránh thứ tự ảo (No False Ordering):**
        - Khác với Label Encoding (gán ID ngẫu nhiên 1, 100...), phương pháp này dùng tỷ lệ thực tế, không làm mô hình hiểu sai về mặt toán học (ví dụ: hiểu nhầm City 100 lớn gấp 100 lần City 1).

    3. **Tăng ngữ nghĩa (Feature Representation):**
        - Biến đổi tên thành phố thành **"Độ phổ biến"**. Giá trị cao đại diện cho các Tech Hub lớn, giá trị thấp là các vùng ven, giúp mô hình học được quy luật hành vi dựa trên quy mô thành phố.

#### **Impute missing**

In [13]:
city_col = data_raw[:, col_index["city"]]

city_filled, city_fill_val = impute_categorical(
    city_col,
    strategy="constant",
    constant_value="Unknown",
)

print("Fill value missing:", city_fill_val)

Fill value missing: Unknown


#### **Frequency encode**

In [14]:
city_freq, city_freq_map = frequency_encode(city_filled)

print("city_freq sample:", city_freq[:10])

city_freq sample: [0.22732018 0.00354943 0.14103769 0.00281867 0.00668128 0.00125274
 0.0441069  0.00668128 0.22732018 0.22732018]


## VI. Feature danh nghĩa (Nominal)

**Các Feature:**
- `gender`
- `company_type`
- `major_discipline`
- `relevent_experience`

*(Lưu ý: `relevent_experience` đã được chuyển sang xử lý dạng Binary 0/1 ở phần khác vì chỉ có 2 giá trị).*

Chiến lược:

1. **Xử lý Missing:**
   - Điền toàn bộ giá trị thiếu bằng nhãn **`"Unknown"`**.
   - **Lý do:** Insight EDA cho thấy nhóm không điền thông tin có hành vi rủi ro cao đặc thù, cần giữ lại nhãn này làm tín hiệu cho mô hình.

2. **Mã hóa (One-Hot Encoding):**
   - Sử dụng hàm `one_hot_encode` với cơ chế **`drop_first=True`**.
   - **Ý nghĩa kỹ thuật:**
     - Biến đổi các nhãn phân loại thành các cột vector số (0/1).
     - Loại bỏ category đầu tiên (khi biến có >1 giá trị) để giảm số chiều dữ liệu và tránh bẫy đa cộng tuyến (Dummy Variable Trap).

#### **Xử lý riêng cho `revelant_experience`**

In [15]:
# Feature này không có missing nên ta bỏ qua phần impute
rel_exp_col = data_raw[:, col_index["relevent_experience"]]
rel_exp_norm = np.char.strip(np.char.lower(rel_exp_col.astype(str)))

print("relevent_experience_binary - trước khi đã xử lí:", rel_exp_norm[:10])

relevent_experience_binary = np.zeros(n_rows, dtype=float)
relevent_experience_binary[rel_exp_norm == "has relevent experience"] = 1.0
relevent_experience_binary[rel_exp_norm == "no relevent experience"] = 0.0

print("relevent_experience_binary - sau khi đã xử lí:", relevent_experience_binary[:10])

relevent_experience_binary - trước khi đã xử lí: ['has relevent experience' 'no relevent experience'
 'no relevent experience' 'no relevent experience'
 'has relevent experience' 'has relevent experience'
 'has relevent experience' 'has relevent experience'
 'has relevent experience' 'has relevent experience']
relevent_experience_binary - sau khi đã xử lí: [1. 0. 0. 0. 1. 1. 1. 1. 1. 1.]


#### **Impute missing**

In [16]:
nominal_onehot_cols = ["gender", "company_type", "major_discipline"]

nominal_filled = {}

for name in nominal_onehot_cols:
    col = data_raw[:, col_index[name]]
    
    filled, fill_val = impute_categorical(
        col,
        strategy="constant",
        constant_value="Unknown",
    )
    nominal_filled[name] = filled
    
    print(f"{name:20s} | fill value missing = {fill_val}")

gender               | fill value missing = Unknown
company_type         | fill value missing = Unknown
major_discipline     | fill value missing = Unknown


#### **One-hot encode**

In [17]:
one_hot_blocks = []
one_hot_feature_names = []

for name in nominal_onehot_cols:
    col_filled = nominal_filled[name]
    
    one_hot, cats = one_hot_encode(col_filled, drop_first=True)
    one_hot_blocks.append(one_hot)
    
    for c in cats:
        one_hot_feature_names.append(f"{name}={c}")

## VII. Feature Engineering (Tạo đặc trưng mới)

Dựa trên các phát hiện chuyên sâu (Deep Dive Insights) từ quá trình EDA, chúng ta tiến hành tạo thêm 4 biến tương tác (Interaction Features) để giúp mô hình bắt được các nhóm hành vi đặc thù mà các biến đơn lẻ không thể hiện được.

**1. `is_startup_veteran` (Cựu binh Startup)**
* **Logic:** `company_type == "Funded Startup"` **VÀ** `last_new_job == ">4"` (nhóm thâm niên cao nhất).
* **Cơ sở (Insight):** EDA chỉ ra rằng nhóm nhân viên làm lâu năm tại các Startup đã gọi vốn có tỷ lệ nghỉ việc thấp kỷ lục (~4.6%). Đây là hiệu ứng "Còng tay vàng" (Golden Handcuffs) – sự gắn kết nhờ quyền lợi cổ phần hoặc vị trí chủ chốt.
* **Tác dụng:** Giúp mô hình nhận diện nhóm **cực kỳ trung thành**.

**2. `brain_drain_risk` (Rủi ro chảy máu chất xám - Senior)**
* **Logic:** `experience > 10 năm` (Senior) **VÀ** `city_development_index < 0.7` (Thành phố kém phát triển).
* **Cơ sở (Insight):** Nhóm chuyên gia cấp cao sống ở thành phố nhỏ có tỷ lệ muốn ra đi rất cao (~40%) để tìm kiếm môi trường xứng tầm hơn ("Cá lớn ao nhỏ").
* **Tác dụng:** Giúp mô hình nhận diện nhóm nhân sự cấp cao có nguy cơ rời đi do yếu tố địa lý.

**3. `is_senior_fulltime` (Senior chuyển hướng sự nghiệp)**
* **Logic:** `experience > 10 năm` (Senior) **VÀ** `enrolled_university == "Full time course"`.
* **Cơ sở (Insight):** Một nhân sự >10 năm kinh nghiệm mà đăng ký học toàn thời gian là dấu hiệu bất thường, thường ám chỉ việc chuẩn bị chuyển đổi nghề nghiệp (Career Pivot). Tỷ lệ nghỉ việc của nhóm này cao vượt trội so với mức trung bình của Senior.
* **Tác dụng:** Giúp mô hình phân biệt được hành vi đi học rủi ro cao của Senior.

**4. `junior_city_flight` (Nguy cơ Junior "bỏ phố nhỏ")**
* **Logic:** `experience < 2 năm` (Junior) **VÀ** `city_development_index < 0.7` (Thành phố kém phát triển).
* **Cơ sở (Insight):** Đây là nhóm có tỷ lệ muốn nghỉ việc **cao nhất toàn bộ tập dữ liệu (~60.6%)**. Người trẻ ở các thành phố nhỏ có xu hướng di cư mạnh mẽ lên các siêu đô thị để tìm kiếm cơ hội học hỏi và Mentor ("Rời đi để lớn").
* **Tác dụng:** Giúp mô hình khoanh vùng chính xác nhóm đối tượng có **xác suất rời đi cao nhất**.

In [18]:
# 1: is_startup_veteran (Cựu binh ở cti Startup)
company_type_filled = nominal_filled["company_type"]
lnj_raw = lnj_num_raw 

idx_gt4 = LAST_NEW_JOB_ORDER.index(">4")

is_funded_startup = (company_type_filled == "Funded Startup")
is_long_since_job_change = (lnj_raw == idx_gt4)

is_startup_veteran = (is_funded_startup & is_long_since_job_change).astype(float)

# 2: brain_drain_risk (Senior ở city_development_index thấp)
exp_raw = exp_num_raw 

idx_11 = EXPERIENCE_ORDER.index("11")
senior_mask = (exp_raw >= idx_11)

cdi_val = cdi_clip  
low_cdi_mask = (cdi_val < 0.7)

brain_drain_risk = (senior_mask & low_cdi_mask).astype(float)

# 3: is_senior_fulltime (Senior đi học Full time)
is_fulltime = (enr_filled == "Full time course")
is_senior_fulltime = (senior_mask & is_fulltime).astype(float)

# 4: junior_city_flight (Junior ở city kém phát triển)
idx_1 = EXPERIENCE_ORDER.index("1")      
junior_mask = (exp_raw <= idx_1)

junior_city_flight = (junior_mask & low_cdi_mask).astype(float)

---
## VIII. Xây dựng Ma trận đặc trưng (Feature Matrix Construction)

Sau khi hoàn tất các bước xử lý riêng lẻ, ta tiến hành tổng hợp các khối dữ liệu để tạo thành không gian đặc trưng cuối cùng.

#### 1. Tổng hợp các thành phần (Feature Blocks)

Ma trận đầu vào được ghép từ 5 nguồn dữ liệu đã qua xử lý:

1.  **Numeric Features (Scaled Z-score):**
    * `city_development_index_z`
    * `training_hours_z`
    * `experience_num_z`
    * `last_new_job_num_z`

2.  **Ordinal Features (Mapped):**
    * `education_level_ord`
    * `company_size_ord`
    * `enrolled_university_ord`

3.  **High-cardinality Features (Frequency Encoded):**
    * `city_freq`

4.  **Nominal Features (One-Hot Encoded):**
    * Gồm các vector nhị phân sinh ra từ: `gender`, `company_type`, `major_discipline`.

5.  **Feature Engineering (Interaction Features):**
    * `is_startup_veteran`
    * `brain_drain_risk`
    * `is_senior_fulltime`
    * `junior_city_flight` *(Đã cập nhật theo chiến lược mới)*


#### 2. Ghép nối ma trận (Concatenation)

* **Đầu vào:** List `feature_blocks` chứa các mảng numpy con.
* **Thao tác:** Sử dụng `np.hstack(feature_blocks)` để ghép nối theo chiều ngang (horizontal).
* **Kết quả:** Ma trận đặc trưng **`X`** có kích thước `(n_samples, total_features)`.
* **Metadata:** Lưu trữ danh sách `feature_names` tương ứng để phục vụ việc phân tích độ quan trọng của biến (Feature Importance) sau này.

#### 3. Chuẩn bị nhãn (Target)

* **Vector `y`:** Trích xuất từ cột `target`.
* **Định dạng:** Ép kiểu về `float` (hoặc `int`) để tương thích với các thư viện Scikit-learn.

In [19]:
feature_blocks = []
feature_names  = []

# 1 Numeric thực (Z-score)
feature_blocks.append(cdi_z.reshape(n_rows, 1))
feature_names.append("city_development_index_z")

feature_blocks.append(train_z.reshape(n_rows, 1))
feature_names.append("training_hours_z")


# 2 Ordinal
feature_blocks.append(exp_num_z.reshape(n_rows, 1))
feature_names.append("experience_num_z")

feature_blocks.append(lnj_num_z.reshape(n_rows, 1))
feature_names.append("last_new_job_num_z")

feature_blocks.append(edu_ord.reshape(n_rows, 1))
feature_names.append("education_level_ord")

feature_blocks.append(csize_ord.reshape(n_rows, 1))
feature_names.append("company_size_ord")

feature_blocks.append(enr_ord.reshape(n_rows, 1))
feature_names.append("enrolled_university_ord")

# 4 City frequency encoding
feature_blocks.append(city_freq.reshape(n_rows, 1))
feature_names.append("city_freq")

# 5 Nominal 
for blk in one_hot_blocks:
    feature_blocks.append(blk)
feature_names.extend(one_hot_feature_names)

feature_blocks.append(relevent_experience_binary.reshape(n_rows, 1))
feature_names.append("relevent_experience_has")

# 6 Feature Engineering
feature_blocks.append(is_startup_veteran.reshape(n_rows, 1))
feature_names.append("is_startup_veteran")

feature_blocks.append(brain_drain_risk.reshape(n_rows, 1))
feature_names.append("brain_drain_risk")

feature_blocks.append(is_senior_fulltime.reshape(n_rows, 1))
feature_names.append("is_senior_fulltime")

feature_blocks.append(junior_city_flight.reshape(n_rows, 1))
feature_names.append("junior_city_flight")

# Ghép X
X = np.hstack(feature_blocks)

# y
target_col = data_raw[:, col_index["target"]]
y = target_col.astype(float)
y_bool = (y == 1.0)

print("X shape        :", X.shape)
print("y shape        :", y.shape)
print("Số feature     :", len(feature_names))

X shape        : (19158, 28)
y shape        : (19158,)
Số feature     : 28


## IX. Kiểm tra & lưu dữ liệu sau preprocessing

Trước khi lưu:

- Kiểm tra không còn `NaN` / `Inf` trong `X`.
- Kiểm tra `X.shape[1] == len(feature_names)`.

Sau đó:

- Ghép `y` + `X` thành một ma trận cuối cùng.
- Ghi ra CSV:
  - Cột đầu: `target`.
  - Các cột sau: toàn bộ feature đã xử lý.
- Lưu vào thư mục `../data/processed` với cùng tên file gốc (`aug_train.csv`).

In [20]:
num_nan = int(np.isnan(X).sum())
num_inf = int(np.isinf(X).sum())
print("NaN trong X:", num_nan)
print("Inf trong X:", num_inf)

assert X.shape[1] == len(feature_names)

print("\nThông tin features sau khi đã được processed:")
print("Số lượng feature:", len(feature_names))
print("Danh sách feature:")
for i, name in enumerate(feature_names):
    print(f"{i:3d} - {name}")

os.makedirs(OUTPUT_DIR, exist_ok=True)
out_path = os.path.join(OUTPUT_DIR, OUTPUT_FILE)

header_out = ["target"] + feature_names

with open(out_path, "w", encoding="utf-8") as f:
    f.write(",".join(header_out) + "\n")
    for i in range(n_rows):
        row_vals = [str(int(y[i]))] + [f"{float(v):.6f}" for v in X[i]]
        f.write(",".join(row_vals) + "\n")

print("Đã lưu file processed tại:", out_path)

NaN trong X: 0
Inf trong X: 0

Thông tin features sau khi đã được processed:
Số lượng feature: 28
Danh sách feature:
  0 - city_development_index_z
  1 - training_hours_z
  2 - experience_num_z
  3 - last_new_job_num_z
  4 - education_level_ord
  5 - company_size_ord
  6 - enrolled_university_ord
  7 - city_freq
  8 - gender=Male
  9 - gender=Other
 10 - gender=Unknown
 11 - company_type=Funded Startup
 12 - company_type=NGO
 13 - company_type=Other
 14 - company_type=Public Sector
 15 - company_type=Pvt Ltd
 16 - company_type=Unknown
 17 - major_discipline=Business Degree
 18 - major_discipline=Humanities
 19 - major_discipline=No Major
 20 - major_discipline=Other
 21 - major_discipline=STEM
 22 - major_discipline=Unknown
 23 - relevent_experience_has
 24 - is_startup_veteran
 25 - brain_drain_risk
 26 - is_senior_fulltime
 27 - junior_city_flight
Đã lưu file processed tại: ../data/processed\aug_train.csv
